<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/managed/vectaraDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vectara Managed Index
In this notebook we are going to show how to use [Vectara](https://vectara.com) with LlamaIndex.

[Vectara](https://vectara.com/) is the trusted AI Assistant and Agent platform which focuses on enterprise readiness for mission-critical applications. 

Vectara provides an end-to-end managed service for Retrieval Augmented Generation or [RAG](https://vectara.com/grounded-generation/), which includes:

1. An integrated API for processing input data, including text extraction from documents and ML-based chunking.

2. The state-of-the-art [Boomerang](https://vectara.com/how-boomerang-takes-retrieval-augmented-generation-to-the-next-level-via-grounded-generation/) embeddings model. Each text chunk is encoded into a vector embedding using Boomerang, and stored in the Vectara internal knowledge (vector+text) store. Thus, when using Vectara with LlamaIndex you do not need to call a separate embedding model - this happens automatically within the Vectara backend.

3. A query service that automatically encodes the query into embeddings and retrieves the most relevant text segmentsthrough [hybrid search](https://docs.vectara.com/docs/api-reference/search-apis/lexical-matching) and a variety of [reranking](https://docs.vectara.com/docs/api-reference/search-apis/reranking) strategies, including a [multilingual reranker](https://docs.vectara.com/docs/learn/vectara-multi-lingual-reranker), [maximal marginal relevance (MMR) reranker](https://docs.vectara.com/docs/learn/mmr-reranker), [user-defined function reranker](https://docs.vectara.com/docs/learn/user-defined-function-reranker), and a [chain reranker](https://docs.vectara.com/docs/learn/chain-reranker) that provides a way to chain together multiple reranking methods to achieve better control over the reranking, combining the strengths of various reranking methods.

4. An option to create a [generative summary](https://docs.vectara.com/docs/learn/grounded-generation/grounded-generation-overview) with a wide selection of LLM summarizers (including Vectara's [Mockingbird](https://vectara.com/blog/mockingbird-is-a-rag-specific-llm-that-beats-gpt-4-gemini-1-5-pro-in-rag-output-quality/), trained specifically for RAG-based tasks), based on the retrieved documents, including citations.

See the [Vectara API documentation](https://docs.vectara.com/docs/) for more information on how to use the API.

The main benefits of using Vectara RAG-as-a-service to build your application are:
* **Accuracy and Quality**: Vectara provides an end-to-end platform that focuses on eliminating hallucinations, reducing bias, and safeguarding copyright integrity.
* **Security**: Vectara's platform provides acess control--protecting against prompt injection attacks--and meets SOC2 and HIPAA compliance.
* **Explainability**: Vectara makes it easy to troubleshoot bad results by clearly explaining rephrased queries, LLM prompts, retrieved results, and agent actions.

## Getting Started

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
!pip install llama-index llama-index-indices-managed-vectara

To get started with Vectara, [sign up](https://vectara.com/integrations/llamaindex) (if you haven't already) and follow our [quickstart guide](https://docs.vectara.com/docs/quickstart) to create a corpus and an API key.

Once you have these, you can provide them as environment variables `VECTARA_CORPUS_KEY`, and `VECTARA_API_KEY`. Make sure your API key has both query and index permissions.

## RAG with LlamaIndex and Vectara

There are a few ways you can index your data into Vectara, including:
1. With the `from_documents()` or `insert_file()` methods of `VectaraIndex`
2. Uploading files directly in the [Vectara console](https://console.vectara.com/)
3. Using Vectara's [file upload](https://docs.vectara.com/docs/rest-api/upload-file) or [document index](https://docs.vectara.com/docs/rest-api/create-corpus-document) APIs
4. Using [vectara-ingest](https://github.com/vectara/vectara-ingest), an open source crawler/indexer project
5. Using one of our ingest integration partners like Airbyte, Unstructured or DataVolo.

For this purpose, we will use a simple set of small documents, so using `VectaraIndex` directly for the ingest is good enough.

Let's ingest the "AI bill of rights" document into our new corpus.

In [ ]:
from llama_index.indices.managed.vectara import VectaraIndex
import requests

url = "https://www.whitehouse.gov/wp-content/uploads/2022/10/Blueprint-for-an-AI-Bill-of-Rights.pdf"
response = requests.get(url)
local_path = "ai-bill-of-rights.pdf"
with open(local_path, "wb") as file:
    file.write(response.content)

index = VectaraIndex()
index.insert_file(
    local_path, metadata={"name": "AI bill of rights", "year": 2022}
)

'ai-bill-of-rights.pdf'

### Running single queries with Vectara Query Engine
Now that we've uploaded the document (or if documents have been uploaded previously) we can go and ask questions directly in LlamaIndex. This activates Vectara's RAG pipeline. 

To use Vectara's internal LLM for summarization, make sure you specify `summary_enabled=True` when you generate the Query engine. Here's an example:

In [ ]:
questions = [
    "What are the risks of AI?",
    "What should we do to prevent bad actors from using AI?",
    "What are the benefits?",
]

In [ ]:
qe = index.as_query_engine(
    n_sentences_before=1,
    n_sentences_after=1,
    summary_enabled=True,
    summary_prompt_name="mockingbird-1.0-2024-07-16",
)
qe.query(questions[0]).response

'The risks of AI include biased data and discriminatory outcomes, opaque decision-making processes, and lack of public trust and understanding of algorithmic systems [1]. These risks can have significant impacts on individuals, communities, and organizations, and can lead to harm, including image-based abuse, which disproportionately affects women [7]. AI systems can also incorrectly penalize individuals, such as drivers, for events beyond their control [7]. To mitigate these risks, ongoing transparency, value-sensitive and participatory design, explanations designed for relevant stakeholders, and public consultation are necessary [1]. Additionally, industry is providing innovative solutions to mitigate risks, including risk assessments, auditing mechanisms, and documentation procedures specific to model assessments [2]. The National Institute of Standards and Technology has also developed an AI Risk Management Framework to help identify and mitigate risks [3].'

If you want the response to be returned in streaming mode, simply set `streaming=True`

In [ ]:
qe = index.as_query_engine(
    n_sentences_before=1,
    n_sentences_after=1,
    summary_enabled=True,
    summary_prompt_name="mockingbird-1.0-2024-07-16",
    streaming=True,
)
response = qe.query(questions[0])

for chunk in response.response_gen:
    print(chunk.delta or "", end="", flush=True)

The risks of AI include biased data and discriminatory outcomes, opaque decision-making processes, and lack of public trust and understanding of algorithmic systems [1]. These risks can have significant impacts on individuals, communities, and organizations, and can lead to harm, including image-based abuse, which disproportionately affects women [7]. AI systems can also incorrectly penalize individuals, such as drivers, for events beyond their control [7]. To mitigate these risks, ongoing transparency, value-sensitive and participatory design, explanations designed for relevant stakeholders, and public consultation are necessary [1]. Additionally, industry is providing innovative solutions to mitigate risks, including risk assessments, auditing mechanisms, and documentation procedures specific to model assessments [2]. The National Institute of Standards and Technology has also developed an AI Risk Management Framework to help identify and mitigate risks [3].ate risks [3].

### Using Vectara Chat

Vectara also supports a simple chat mode. In this mode the chat history is maintained by Vectara and so you don't have to worry about it. To use it simple call `as_chat_engine`.

(Chat mode always uses Vectara's summarization so you don't have to explicitly specify `summary_enabled=True` like before)

In [ ]:
ce = index.as_chat_engine(n_sentences_before=1, n_sentences_after=1)

In [ ]:
for q in questions:
    print(f"Question: {q}\n")
    response = ce.chat(q).response
    print(f"Response: {response}\n")

Question: What are the risks of AI?

Response: The risks of AI include potential biases in data leading to discriminatory outcomes, opaque decision-making processes, lack of public trust, and understanding of algorithmic systems. Mitigation strategies involve ongoing transparency, participatory design, explanations for stakeholders, and public consultation to ensure safety and efficacy. Risks also encompass the impact on individuals' rights, opportunities, and communities, as well as the potential for misuse of AI systems. Additionally, there are concerns about image-based abuse disproportionately affecting women and instances where AI systems incorrectly penalize individuals due to errors or external factors beyond their control.

Question: What should we do to prevent bad actors from using AI?

Response: To prevent bad actors from using AI, we should implement comprehensive measures. These include adhering to principles such as ensuring AI is lawful, respectful, accurate, safe, secur

Of course streaming works as well with Chat:

In [ ]:
ce = index.as_chat_engine(
    n_sentences_before=1, n_sentences_after=1, streaming=True
)

In [ ]:
response = ce.stream_chat("Will artificial intelligence rule the government?")
for chunk in response.chat_stream:
    print(chunk.delta or "", end="", flush=True)

Artificial intelligence will not rule the government. The government has established principles and guidelines for the ethical use of AI, ensuring that it is used responsibly, lawfully, and in alignment with the nation's values. These principles emphasize safety, accountability, transparency, and effectiveness in the deployment of AI within the federal government [1] [2]. Additionally, there are specific considerations for sensitive areas like law enforcement and national security, where special requirements and oversight mechanisms are in place to safeguard against misuse [3]. The government is focused on promoting equity, fairness, civil rights, and racial justice through the use of AI, guided by principles that protect the American public and uphold democratic values [5]. The establishment of councils and offices dedicated to AI advancement within government agencies further demonstrates a structured and controlled approach to AI integration, ensuring it serves the public interest [

### Agentic RAG

Vectara also has its own package, [vectara-agentic](https://github.com/vectara/py-vectara-agentic), built on top of many features from LlamaIndex to easily implement agentic RAG applications. It allows you to create your own AI assistant with RAG query tools and other custom tools, such as making API calls to retrieve information from financial websites. You can find the full documentation for vectara-agentic [here](https://vectara.github.io/vectara-agentic-docs/).

Let's create a ReAct Agent with a single RAG tool using vectara-agentic (to create a ReAct agent, specify `VECTARA_AGENTIC_AGENT_TYPE` as `"REACT"` in your environment).

Vectara does not yet have an LLM capable of acting as an agent for planning and tool use, so we will need to use another LLM as the driver of the agent resoning.

In this demo, we are using OpenAI's GPT4o. Please make sure you have `OPENAI_API_KEY` defined in your environment or specify another LLM with the corresponding key (for the full list of supported LLMs, check out our [documentation](https://vectara.github.io/vectara-agentic-docs/introduction.html#try-it-yourself) for setting up your environment).

In [ ]:
!pip install -U vectara-agentic

In [ ]:
from vectara_agentic.agent import Agent
from IPython.display import display, Markdown

agent = Agent.from_corpus(
    tool_name="query_ai",
    data_description="AI regulations",
    assistant_specialty="artificial intelligence",
    vectara_reranker="mmr",
    vectara_rerank_k=50,
    vectara_summary_num_results=5,
    vectara_summarizer="mockingbird-1.0-2024-07-16",
    verbose=True,
)

response = agent.chat(
    "What are the risks of AI? What are the benefits? Compare and contrast and provide a summary with arguments for and against from experts."
)

display(Markdown(response))

Failed to set up observer (No module named 'phoenix.otel'), ignoring
> Running step ed1749a3-91c3-4477-8415-cf5646402b73. Step input: What are the risks of AI? What are the benefits? Compare and contrast and provide a summary with arguments for and against from experts.
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: query_ai
Action Input: {'query': 'risks and benefits of AI, expert opinions'}


Query warning(s) exceeded_max_input_length_fcs


Observation: 
                    Response: '''According to expert opinions, the risks of AI include biased data and discriminatory outcomes, opaque decision-making processes, and lack of public trust and understanding of algorithmic systems [1]. To mitigate these risks, experts emphasize the importance of ongoing transparency, value-sensitive and participatory design, explanations designed for relevant stakeholders, and public consultation [1]. Additionally, industry is providing innovative solutions to mitigate risks to the safety and efficacy of AI systems, including risk assessments, auditing mechanisms, and documentation procedures [3]. The National Institute of Standards and Technology (NIST) is developing a risk management framework to better manage risks posed to individuals, organizations, and society by AI [3]. Furthermore, the White House Office of Science and Technology Policy has led a year-long process to seek input from people across the country on the issue of algorithm

The risks and benefits of AI are widely discussed among experts, and there are compelling arguments on both sides.

**Risks of AI:**
1. **Bias and Discrimination**: AI systems can perpetuate and even amplify biases present in the data they are trained on, leading to discriminatory outcomes. This is a significant concern in areas like hiring, law enforcement, and lending.
2. **Opaque Decision-Making**: Many AI systems, particularly those based on deep learning, operate as "black boxes," making it difficult to understand how decisions are made. This lack of transparency can erode trust.
3. **Public Trust and Understanding**: There is often a gap in understanding how AI systems work, which can lead to mistrust and fear among the public.
4. **Safety and Efficacy**: Ensuring that AI systems are safe and effective is a challenge, particularly as they become more complex and autonomous.

To address these risks, experts recommend transparency, participatory design, stakeholder-specific explanations, and public consultation. Industry solutions include risk assessments, auditing mechanisms, and documentation procedures. The National Institute of Standards and Technology (NIST) is also developing a risk management framework to manage these risks effectively.

**Benefits of AI:**
1. **Efficiency and Productivity**: AI can automate routine tasks, leading to increased efficiency and productivity in various sectors, from manufacturing to healthcare.
2. **Innovation**: AI drives innovation by enabling new products and services, such as personalized medicine and autonomous vehicles.
3. **Data Analysis**: AI excels at analyzing large datasets, uncovering patterns and insights that would be impossible for humans to detect.
4. **Improved Decision-Making**: By providing data-driven insights, AI can enhance decision-making processes in fields like finance, healthcare, and logistics.

In summary, while AI presents significant risks, particularly related to bias, transparency, and trust, it also offers substantial benefits in terms of efficiency, innovation, and data analysis. Balancing these risks and benefits requires careful consideration, ongoing transparency, and robust risk management frameworks, as highlighted by experts and initiatives like the Blueprint for an AI Bill of Rights [1][3][4].